In [ ]:
from hackathon import surrogate_model
import argparse
import numpy as np
import pandas as pd
import os
from hackathon import search_model
from hackathon.src.datasets.data_loader import load_data
import hackathon.src.dynamic_pipeline as dynamic_pipeline


In [2]:
def concrete():
    df = pd.read_csv('./hackathon/data/concrete.csv')

    target = 'strength'
    user_request_target = 40.0

    control_name = ['cement', 'slag', 'ash', 'water', 'superplastic', 'coarseagg', 'fineagg', 'age']
    importance={'cement': 1,
                    'water': 2,
    }
    optimize={'cement': 'minimize',
                'water': 'minimize',
    }
    return df, target, user_request_target, control_name, importance, optimize 

In [3]:
def dvm():
    df = pd.read_csv('./hackathon/data/dvm_merged_df.csv')

    target = 'Annual_revenue'
    user_request_target = 2485797880

    control_name = ['Entry_price', 'Fuel_type', 'Bodytype', 'Top_speed', 'Gearbox']
    # control_name = [col for col in df.columns if col != target]
    importance={'Entry_price': 1,
                    'Top_speed': 2,
    }
    optimize={'Entry_price': 'minimize',
                'Top_speed': 'minimize',
    }
    return df, target, user_request_target, control_name, importance, optimize 

In [4]:
def ecommerce():
    df = pd.read_csv('./hackathon/data/ecommerce_sampled_10000.csv')

    target = 'revenue'
    user_request_target = 50000000

    control_name = ['discount_rate']
    # control_name = [col for col in df.columns if col != target]
    importance={'discount_rate': 1,
    }
    optimize={'discount_rate': 'minimize',
    }
    return df, target, user_request_target, control_name, importance, optimize 

In [5]:
def melb():
    df = pd.read_csv('./hackathon/data/melb_data.csv')

    target = 'Price'
    user_request_target = 7000000

    control_name = ['Suburb', 'Rooms', 'Landsize']
    # control_name = [col for col in df.columns if col != target]
    importance={'Landsize': 1,
            'Rooms': 2,
    }
    optimize={'Landsize': 'minimize',
            'Rooms': 'minimize',
    }
    return df, target, user_request_target, control_name, importance, optimize 

In [6]:
def employee():
    df = pd.read_csv('./hackathon/data/employee_attrition_dataset.csv')

    target = 'Attrition'
    user_request_target = 'No'

    control_name = ['Job_Role', 'Hourly_Rate', 'Average_Hours_Worked_Per_Week', 'Relationship_with_Manager', 'Job_Involvement', 'Number_of_Companies_Worked']
    # control_name = [col for col in df.columns if col != target]
    importance={'Hourly_Rate': 1,
                'Average_Hours_Worked_Per_Week': 2,
    }
    optimize={'Hourly_Rate': 'minimize',
            'Average_Hours_Worked_Per_Week': 'maximize',
    }
    return df, target, user_request_target, control_name, importance, optimize 

In [7]:
df, target, user_request_target, control_name, importance, optimize = concrete() # concrete(), dvm(), ecommerce(), melb(), employee()
model = 'catboost' # 'catboost','tabpfn'

In [ ]:
ret_dict = dynamic_pipeline.preprocess_dynamic(df.copy())
ret_dict[0].to_csv('./temp.csv', index=False)

In [ ]:
# if target not in ret_dict['processed_df'].columns: raise Exception('unexpected target')
args = argparse.Namespace(
    target = [target],
    data_path='./temp.csv',
    model=model,
    prj_id = 40,
    seed=40
)
surrogate_model.main(args, ret_dict[-1])

In [10]:
if target not in df.columns: raise Exception('unexpected target')
control_range = df[control_name]
control_range = {col: (df[col].min(), df[col].max()) for col in control_range}

In [ ]:
print(target, df[target].min(),'~',df[target].max())

In [12]:
df_scaled = load_data(args.data_path)
X = df_scaled.drop(columns=args.target)
y = df_scaled[args.target]
x_col_list = X.columns.tolist()

X_train,y_train = X.to_numpy(),y.to_numpy()


In [13]:
scalers = ret_dict[-1]


In [14]:
from hackathon.src.datasets.data_loader import load_data

In [15]:
for l in [control_name, importance, optimize]:
    for c in l:
        if c not in ret_dict[0].columns:
            raise Exception(f'{c} is unexpected col_name')
args = argparse.Namespace(
    model=model,
    search_model='k_means',
    data_path='./temp.csv',
    control_name=control_name,
    control_range= control_range,
    target=[target],
    importance=importance,
    optimize=optimize,
    prj_id=40,
    seed=40,
    user_request_target=[user_request_target],
    model_path='./temp/surrogate_model/model',
    user_request_idx=10
)

In [ ]:
# %%time 
x_opt = search_model.main(args,ret_dict[-1])

In [ ]:
df

In [ ]:
x_opt

In [ ]:
x_opt.iloc[1]